# Bayesian Neural Network with the  MNIST dataset

### Dependencies:
    tensorflow
    tf.keras 
    numpy
    matplotlib
    pandas
    seaborn
    edward

### Dataset:
    - the MNIST dataset of handwritten digits: http://yann.lecun.com/exdb/mnist/
    - Training set: 60,000 images, Testing set: 10,000 images
    - 10 categories: {0,1,..9}
    - 28x28 image = 784 features 
    - nonMNIST images can be found here: http://yaroslavvb.com/upload/notMNIST/ or davidflanagan/notMNIST-to-MNIST.git 

    
### Model:
    Task: classify the handwritten MNIST digits into one of the 10 classes {0,1,2,...9} and give a measure of the uncertainty of the classificatin. 
    Model: soft-max regression 
    Likelihood function: Categorical likelihood function - to quantify the probability of the observed data given a set of parameters, weights and biases in this case. The Categorical distrubution is also known as the Multinoulli distribution. 
    Infer the posterior using Variational Inference - minimize the KL divergence between the the true posterior and approximating distributions 
    We evaluate the model with a set of predictions and their accuracies, instead of a single prediction, as per regular NNs.



In [11]:
import tensorflow as tf 
from tensorflow import keras
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from tensorflow.examples.tutorials.mnist import input_data

import seaborn as sns
from edward.models import Categorical, Normal
import edward as ed


### 1. Import data sets - MNIST 

In [13]:
# Load the data using TensorFlow methos
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


### 2. Build the model 

In [ ]:
# Use Edward to place priors on the weights and biases
ed.set_seed(314159)
N = 100 # number of images in a minibatch
D = 784 # number of features
K = 10 # number of classes

# Create a placeholder for the data in minibatches in a TensorFlow graph
x = tf.placeholder(tf.float32, [None, D])

# place a normal Gaussian (0,1) prior on the weights and biases 
w = Normal(loc=tf.zeros([D, K]), scale=tf.ones([D,K]))
b = Normal(loc=tf.zeros(K), scale=tf.ones(K))

# Categorical likelihood for classification
y = Categorical(tf.matmul(x,w)+b)


### 3. Variational Inference 

In [12]:
# The posterior is not calculated directly, 
# its practically too computationally expensive 
# but we use a Variational Inference technique:
# minimize the KL divergence between the true posterior 
# and the approximating distribution

# Contruct the approximate distributions q(w) for weights and q(b) for biases
# assume Normal distributions
qw = Normal(loc=tf.Variable(tf.random_normal([D,K])),scale=tf.nn.softplus(tf.Variable(tf.random_normal([D,K]))))
qb = Normal(loc=tf.Variable(tf.random_normal([K])), scale=tf.nn.softplus(tf.Variable(tf.random_normal([K]))))

# Create a placeholder for the labels
y_ph = tf.placeholder(tf.int32, [N])

# Define the Variational Inference technique - 
# minimize the KL divergence 
inference = ed.KLqp({w: qw, b: qb}, data={y:y_ph})

# Initliaze the inference variables
inference.initialize(n_iter=5000, n_print=100, scale={y: float(mnist.traing.num_examples) / N})

#Load a tensorflow session, initialize all the variables
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()


### 4. Train the model

In [ ]:
# Start interations - begin training 
# load the data in minibatches and update VI infernce using each new batch
for _ in range(inference.n_iter):
    X_batch, Y_batch = mnist.train.next_batch(N)
    Y_batch = np.argmax(Y_batch, axis=1)
    info_dict = inference.update(feed_dict={x: X_batch, y_ph: Y_batch})
    #inference.print_progress(info_dict)


### 5. Evaluate the model

In [ ]:
# Evaluate with a set of predictions and accuracies
# draw 100 samples from the posterior distribution, 
# and check performance on each sample

# Load the test images and convert to a single label
X_test = mnist.test.images
Y_test = np.argmax(mnist.test.labels, axis=1)

# Generate samples the posterior and store them
n_samples = 100
prob_lst = []
samples = []
w_samples = []
b_samples = []

for _ in range(n_samples):
    w_samp = qw.sample()
    b_samp = qb.sample()
    w_samples.append(w_samp)
    b_samples.append(b_samp)
    
    #compute the probability of each class for each (w,b) sample
    prob = tf.nn.softmax(tf.matmul(X_test, w_samp)+ b_samp)
    prob_lst.append(prob_eval())
    sample = tf.concat([tf.reshape(w_sampe, [-1]), b_samp],0)
    samples.append(sample.eval())
    

In [ ]:
# Compute the accuracy of the model
accy_test=[]
for prob in prob_lst:
    y_trn_prd = np.argmax(prob, axis=1).astype(np.float32)
    acc = (y_trn_prd == Y_test).mean()*100
    acc_test.append(acc)
    
plt.hist(accy_test)
plt.title("Histogram of prediction accuracies in the MNIST test data")
plt.xlabel("Accuracy")
plt.ylabel("Frequency")

In [ ]:
# model averaging will give us the equivalent of a classical machine learning model
Y_pred = np.argmax(np.mean(prob_lst,axis=0), axis=1)
print("Accuracy in predicting the test data = ", (Y_pred == Y_test).mean()*100)


In [ ]:
# Look at the first image from the test data and its label
test_image = X_test[0:1]
test_label = Y_test[0]
print("Truth = ", test_label)

pixels = test_image.reshape((28,28))
plt.imshow(pixels,cmap='Blues')

In [ ]:
# Check what the model predicts for each (w,b) sample from the posterior
sing_img_probs = []
for w_samp,b_samp in zip(w_samples, b_samples):
    prob = tf.nn.softmax(tf.matmul(X_test[0:1],w_samp)+b_samp)
    sing_img_probs.append(prob.eval())

In [ ]:
# Create a histogram of these predictions.
plt.hist(np.argmax(sing_img_probs,axis=2),bins=range(10))
plt.xticks(np.arange(0,10))
plt.xlim(0,10)
plt.xlabel("Accuracy of the prediction of the test digit")
plt.ylabel("Frequency")
